<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Complaints Analysis Integration with Customer360
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;'><b>Introduction:</b></p>
<p style='font-size:16px;font-family:Arial;'>Complaints Analysis Integration with Customer360 is a comprehensive approach to managing customer complaints and feedback within the framework of a Customer 360-degree view using <b>Teradata Vantage</b> and <b>Google Gemini</b>. This integration aims to provide a seamless and personalized customer experience by leveraging data from various sources, including CRM systems, marketing platforms, and social media.</p> <p style='font-size:16px;font-family:Arial;'>The key components of this integration include:</p> 

<ol style='font-size:16px;font-family:Arial;'> <li><b>Customer 360 Data Manager</b>: Responsible for managing and maintaining a comprehensive view of customer data, including collection, integration, and analysis from multiple sources.</li> <li><b>Complaints Management Dashboard</b>: Analyzes customer complaints, providing insights into complaint volume, trends, and resolution progress.</li> <li><b>Customer Insights</b>: Tools for gaining insights into customer behavior and preferences, enabling targeted marketing campaigns and informed business decisions.</li> </ol> <p style='font-size:16px;font-family:Arial;'>The benefits of this integration include:</p> <ol style='font-size:16px;font-family:Arial;'> <li><b>Improved Customer Experience</b>: By integrating complaints analysis with Customer 360, businesses can address customer complaints more effectively, leading to increased customer satisfaction and loyalty.</li> <li><b>Data-Driven Decision Making</b>: The integration provides a centralized platform for analyzing customer data, enabling businesses to make informed decisions about product development, marketing strategies, and customer engagement.</li> <li><b>Enhanced Customer Insights</b>: The comprehensive view of customer data allows businesses to better understand customer needs and preferences, leading to more targeted and effective marketing efforts.</li> </ol> 


<p style='font-size:16px;font-family:Arial;'>By integrating complaints analysis with Customer 360, businesses can create a more comprehensive and personalized customer experience, driving business growth and customer satisfaction.</p> 

<p style = 'font-size:16px;font-family:Arial;'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial;'>
     <li>Configuring the environment</li>
    <li>Connect to Vantage</li>
    <li>Setup API key for Google Gemini</li>
    <li>Using LLM for Sentiment Analysis, Topic Modelling and Complaint Summerization</li>
    <li>Integrated data with customer 360</li>
    <li>Cleanup</li>
</ol>

<hr style='height:2px;border:none;'>
<b style = 'font-size:20px;font-family:Arial;'>1. Configuring the environment</b>

<p style = 'font-size:18px;font-family:Arial;'><b>1.1 Install the required libraries</b></p>

In [ ]:
%%capture
!pip install -r requirements.txt --quiet

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;'><b>Note: </b><i>Please restart the kernel after executing these two lines. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial;'><b>1.2 Import the required libraries</b></p>

<p style = 'font-size:16px;font-family:Arial;'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
import pandas as pd
from tqdm import tqdm
from teradataml import *

# GenAI libs
import google.generativeai as genai

display.max_rows = 5
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_rows', 5)

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>2. Connect to Vantage</b>

<p style = 'font-size:18px;font-family:Arial;'><b>2.1 Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)
execute_sql('''SET query_band='DEMO=Complaint_Analysis_Customer360.ipynb;' UPDATE FOR SESSION;''')

<p style = 'font-size:16px;font-family:Arial;'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:1px;border:none;'>

<p style = 'font-size:18px;font-family:Arial;'><b>2.2 Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_ComplaintAnalysis_cloud');"        # Takes 1 minute
# %run -i ../run_procedure.py "call get_data('DEMO_ComplaintAnalysis_local');"        # Takes 2 minutes

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>3. Setup API key for Google Gemini</b>
<p style = 'font-size:16px;font-family:Arial;'>Please enter the Google API Key, if you don't have one, please get it from <a href = 'https://ai.google.dev/gemini-api/docs/api-key'>here</a></p>

In [ ]:
GOOGLE_API_KEY = getpass.getpass(prompt = 'Please enter GOOGLE_API_KEY: ')
genai.configure(api_key = GOOGLE_API_KEY)

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial;'>3.1. Define the Gemini model and Prompt</b>
<p style = 'font-size:16px;font-family:Arial;'>The following section defines the type of Gemini model used. Here we use <b>gemini-1.5-flash</b></p>

In [ ]:
from google.generativeai.types import HarmCategory, HarmBlockThreshold

model = genai.GenerativeModel(
    model_name = "models/gemini-2.5-flash"
)

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>4. Using LLM for Sentiment Analysis, Topic Modelling and Complaint Summarization</b>

<p style = 'font-size:16px;font-family:Arial;'>Sentiment Analysis, Topic Modelling and Complaint Summarization using Large Language Models (LLMs) revolutionizes the way we understand and categorize vast collections of text data. LLMs excel in understanding the semantics and context of words, enabling sophisticated topic modeling techniques.</p>

<p style='font-size:16px;font-family:Arial;'>Sentiment Analysis Using Large Language Models (LLMs) is a cutting-edge approach to understanding customer opinions and emotions expressed through text-based data. This advanced technique leverages the capabilities of LLMs to accurately identify and categorize sentiment as positive, negative, or neutral, providing businesses with valuable insights into customer perceptions and preferences.</p>

<p style = 'font-size:16px;font-family:Arial;'>LLMs can generate coherent topics without needing predefined categories, making them ideal for exploratory analysis of diverse datasets. Moreover, their ability to capture subtle nuances in language allows for more precise topic identification, even in noisy or ambiguous texts.</p>

<p style = 'font-size:16px;font-family:Arial;'><b>Reasoning with a Chain of Thought</b>: Imagine you're trying to solve a problem. With a large language model, you start with an initial idea or question. Then, you use the model's capabilities to explore related concepts, gradually connecting them together. Each step builds upon the previous one, leading you closer to understanding or solving the problem. It's like putting together puzzle pieces, one by one, until you see the whole picture.</p>

In [ ]:
customer_data = DataFrame(in_schema('DEMO_ComplaintAnalysis', 'Customer_360_Details'))
customer_data

In [ ]:
complaints_data = DataFrame(in_schema('DEMO_ComplaintAnalysis', 'Customer_360_Complaints'))
complaints_data

In [ ]:
pd_df = complaints_data.to_pandas()
pd_df['Sentiment'] = ""
pd_df['Topic'] = ""
pd_df['Summary'] = ""
pd_df['Strategy'] = ""


pd_df

In [ ]:
# Sentiment
for i in tqdm(range(len(pd_df))):
    try:
        prompt = f'''
        User prompt: 
        The following is text from a review:

        “{pd_df['consumer_complaint_narrative'][i]}”

        Categorize the review as one of the following:

        Positive
        Negative
        Neutral

        - Important: Do not add any formatting into the output.
        - Return just one of the above options, Do not return explanation
        '''
        output = model.generate_content([prompt])
        sentiment = output.candidates[0].content.parts[0].text

        pd_df['Sentiment'][i] = sentiment
    except:
        pass

In [ ]:
# Topic
for i in tqdm(range(len(pd_df))):
    try:
        prompt = f'''
        User prompt:
        The following is text from a complaint:

        “{pd_df['consumer_complaint_narrative'][i]}”

        Identify the topic of the complaint and categorize into one of the following topics. Only output one of the following options:

        - Mortgage Application
        - Payment Trouble
        - Mortgage Closing
        - Report Inaccuracy
        - Payment Struggle

        - Important: Do not add any formatting into the output. For example **Mortgage Application** or **Report Inaccuracy** refrain from such formating in the response.
        - Return just one of the above options
    '''

        output = model.generate_content([prompt])
        topic = output.candidates[0].content.parts[0].text

        pd_df['Topic'][i] = topic
    except:
        pass

In [ ]:
# Summary
for i in tqdm(range(len(pd_df))):
    try:
        prompt = f'''
            The following is text from a Bank Review:
            “{pd_df['consumer_complaint_narrative'][i]}”
            Summarize the Bank Review in one sentence
        '''

        output = model.generate_content([prompt])
        summary = output.candidates[0].content.parts[0].text

        pd_df['Summary'][i] = summary
    except:
        pass

In [ ]:
# Strategy
for i in tqdm(range(len(pd_df))):
    try:
        prompt = f'''
        User prompt:
        The following is text from a complaint:

        “{pd_df['consumer_complaint_narrative'][i]}”

        Suggest the best course of action for the bank from the following:

        - Wealth Manager to contact customer immediately
        - Send Policy Letter from Mortgage Servicing
        - Send Policy Letter from Executive Office
        - Mortgage Banker to follow-up with Title Company for documentation and contact customer
        - Branch Manager to contact customer immediately

        - Important: Do not add any formatting into the output.
        - Return just one of the above options
    '''

        output = model.generate_content([prompt])
        strategy = output.candidates[0].content.parts[0].text

        pd_df['Strategy'][i] = strategy
    except:
        pass

In [ ]:
pd_df['Sentiment'] = pd_df['Sentiment'].apply(lambda x: x.strip())
pd_df['Topic'] =  pd_df['Topic'].apply(lambda x: x.strip())
pd_df['Summary'] = pd_df['Summary'].apply(lambda x: x.strip())
pd_df['Strategy'] = pd_df['Strategy'].apply(lambda x: x.strip())


combined_df = customer_data.to_pandas().join(pd_df)

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>5. Integrated data with customer 360</b>
<p style = 'font-size:16px;font-family:Arial;'>The following is an example of the output from LLM integrated with existing customer360 data. Please scroll to the right to see all the columns.</p>

In [ ]:
pd.set_option('display.max_colwidth', None)
combined_df[["complaint_id","Customer_ID","Sentiment","Topic","Summary", "Strategy"]]

<p style = 'font-size:16px;font-family:Arial;'>Now the results can be saved back to Vantage.</p>

In [ ]:
combined_df.drop(columns=['state'], inplace=True)

In [ ]:
copy_to_sql(df = combined_df, table_name = 'Customer_360_Complaints_analysis', if_exists = 'replace', primary_index="complaint_id")

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial;'>6. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_ComplaintAnalysis');"        # Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial;'>Dataset:</b>
<br>
<br>
<p style='font-size: 16px; font-family: Arial; '>The dataset is sourced from <a href='https://www.consumerfinance.gov/data-research/consumer-complaints/'>Consumer Financial Protection Bureau</a></p>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025. All Rights Reserved
        </div>
    </div>
</footer>